In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [0]:
%sql
select * from otc.bronze.src_shipments

In [0]:
watermark_ts = spark.sql("select * from otc.silver.watermark where table_name='shipments' ").select('watermark_ts').collect()[0][0]

eligible_shipments_records = spark.sql(f"""
                                      select * from otc.bronze.src_shipments
                                      where ingest_ts > TIMESTAMP '{watermark_ts}'
                                      """)

# Dedup in silver
eligible_shipments_records = eligible_shipments_records.withColumn('SequenceOfRecord', F.row_number().over( Window.partitionBy('shipment_event_id').orderBy( F.desc('ingest_ts') ) ) )\
    .filter( F.col('SequenceOfRecord') == 1)\
        .drop('SequenceOfRecord')

eligible_shipments_records.display()
eligible_shipments_records.createOrReplaceTempView("eligible_shipments_records")

In [0]:
%sql
describe otc.silver.shipments

In [0]:
%sql

INSERT INTO otc.silver.shipments (shipment_event_id, order_id, shipment_ts, shipment_status, ingest_ts)
SELECT shipment_event_id, order_id, shipment_ts, shipment_status, ingest_ts FROM eligible_shipments_records

In [0]:
%sql
UPDATE otc.silver.watermark
  SET watermark_ts = (select max(ingest_ts) from otc.silver.shipments)
  WHERE table_name = 'shipments'

In [0]:
%sql
select * from otc.silver.shipments

In [0]:
%sql
select * from otc.silver.watermark